In [3]:
import rasterio
import shapely
from shapely.geometry import (Polygon, shape)
from rasterio.mask import mask
from pyproj import Proj, transform
import geopandas as gpd

In [4]:
#Function to extract the rooftop image from the source image and save it in '.tif' format
src = rasterio.open('/home/amit/guatemala/mixco_1_and_ebenezer/mixco_1_and_ebenezer_ortho-cog.tif')
aoi_geojson = gpd.read_file('/home/amit/guatemala/mixco_1_and_ebenezer/train-mixco_1_and_ebenezer.geojson')
for i in range(aoi_geojson.shape[0]):
    cord = aoi_geojson['geometry'][i].bounds
    inProj = Proj(init='epsg:4326')
    outProj = Proj(init='epsg:32616')
    (x1,y1,x2,y2) = cord
    x1_new,y1_new = transform(inProj,outProj,x1,y1)
    x2_new,y2_new = transform(inProj,outProj,x2,y2)
    ar_new = shapely.geometry.box(x1_new, y1_new, x2_new, y2_new, ccw=True)
    crop, cropTransform = mask(src, [ar_new], crop=True)
    meta = src.meta.copy()
    meta.update({"transform":cropTransform, "height":crop.shape[1], "width":crop.shape[2]})

    with rasterio.open('Processed data/'+aoi_geojson['id'][i]+'.tif', 'w', **meta) as dst:
        dst.write(crop)

In [5]:
import cv2

img = cv2.imread('Processed data/7a28207c.tif')
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
img.shape

(270, 261, 3)